In [5]:
import warnings
warnings.filterwarnings('ignore')
import json, os
import numpy as np
from monty.json import MontyDecoder, MontyEncoder
from copy import deepcopy
from scipy.spatial import distance_matrix
import sys
sys.path.append('..')

from phasemapy.dataio import InstanceData
from phasemapy.parser import ICDDEntry
from phasemapy.solver import Phase, Sample,Texture

chemsys = ['V', 'Mn', 'Nb']
oxide_system = True
photon_e = 13e3
max_q_shift = 0.02
resample_density = 1000
initial_alphagamma = 0.1
SUM_NORM = 6000
loss_weight = {'xrd_loss': 6.0, 'comp_loss': 2.0, 'entropy_loss': 0.01}

In [6]:
instance_data = InstanceData.from_file('../data/instance_file_24297_NbMnVO_v02.txt', chemsys, photon_e)
instance_data = instance_data.resample_xrd(resample_density)
instance_data.renormalize(norm=SUM_NORM)
instance_data.normalize()

In [7]:
samples = []
for i in range(instance_data.sample_num):
    solution_file = f'solution/samples{i}.json'
    with open(solution_file) as f:
        sample = json.load(f, cls=MontyDecoder)
    if sample.sample_id !=i:
        print (i)    
    samples.append(sample)

In [8]:
Sample.solution_statistics(samples)

      entry_id         tot       max       min  count     names
0  04-014-2995  147.128443  1.000000  0.074725    274   Nb2MnO6
1  04-005-5308   36.459623  1.000000  0.443767     78    V2MnO6
2  00-052-1266   34.724134  0.702429  0.351906    164   Mn2V2O7
3  04-007-1841   34.578645  0.509747  0.017255    163     Mn3O4
4        22736   26.220035  0.739389  0.180791     87   Nb5VO15
5  00-041-1442   17.749090  0.589172  0.039922    113     Mn2O3
6  04-023-0598    9.621416  0.641599  0.397653     29  Nb(VO4)O
7  00-052-1265    6.969591  0.395701  0.030235     85   Mn2V2O7
8  04-007-0398    3.146837  0.158580  0.158580     54      V2O5
9  00-068-0146    0.402186  0.106069  0.045530      6  Nb8.4O21


In [23]:
# for sample in samples:    
#     solution_file = f'solution/samples{sample.sample_id}.json'
# #     sample.refine_one_by_one()
#     sample.plot(perphase=True)
#     with open(solution_file, 'w') as f:
#         json.dump(sample, f, cls=MontyEncoder)
#         sample.plot(perphase=True, saveplot=f'solution_figures/sample_{sample.sample_id}.pdf')

In [4]:
# comp_dist = distance_matrix(instance_data.sample_comp, instance_data.sample_comp)
# nn_list = {i: np.where((comp_dist[i] < 0.5) & (comp_dist[i] > 0))[0] for i in range(instance_data.sample_num)}
# for sample in samples:
#     if True:
#         # if min([count_act[_.entry_id] for _ in sample.entries])<5.0:
#         if sample.R > 0.12:
#             print(sample.sample_id, sample.loss(loss_weight))
#             candidate_entries = []
#             for i in nn_list[sample.sample_id]:
#                 candidate_entries += samples[i].entries
#             candidate_entries = list(set(candidate_entries))
#             solution = []
#             for e in candidate_entries:
#                 phase = Phase.from_entry_and_instance_data(e, 1 / len(candidate_entries), instance_data)
#                 solution.append(phase)
    
#             new_sample = deepcopy(sample)
#             new_sample.solution = solution
#             new_sample.refine_one_by_one()
#             new_sample.refine_all_fractions()   
#             new_sample.update_solution(0.01, 0.2999, new_sample.max_q_shift)            
#             new_sample = new_sample.optimize(num_epoch=500, print_prog=True, loss_weight=loss_weight)
#             new_sample.update_solution(0.01, 0.2999, new_sample.max_q_shift)  
           
#             new_sample.refine_one_by_one() 
#             new_sample.refine_all_fractions()          
#             new_sample.update_solution(0.05, 0.2999, new_sample.max_q_shift) 
#             new_sample.print_solution()           
#             if new_sample.loss(loss_weight) <= sample.loss(loss_weight):
#                 sample.print_solution()
#                 new_sample.print_solution()                
#                 new_sample.plot(perphase=True)
#                 samples[sample.sample_id] = new_sample
#                 solution_file = f'solution/samples{sample.sample_id}.json'
#                 with open(solution_file, 'w') as f:
#                     json.dump(samples[sample.sample_id], f, cls=MontyEncoder)

In [5]:
# samples = []
# for i in range(instance_data.sample_num):
#     solution_file = f'solution/samples{i}.json'
#     with open(solution_file) as f:
#         sample = json.load(f, cls=MontyDecoder)
#     if sample.sample_id !=i:
#         print (i)    
#     samples.append(sample)

In [ ]:
texture_groups = {}
new_samples = []
for i,sample in enumerate(samples):
    iter_num = 1
    print(i)
    sample_texture = Texture(sample,TC_cutoff=1.2)
    while  iter_num <= len(sample.solution):
        if len(sample_texture.preferred_orientation ) != 0:
            print(sample_texture.entry.name)
            print(sample_texture.preferred_orientation)
            if sample_texture.entry.chemical_formula in texture_groups.keys():
                texture_group_one = texture_groups[f'{sample_texture.entry.chemical_formula}_{sample_texture.preferred_orientation}']
            else:
                try:
                    texture_group_one = sample_texture.get_texture_group()
                    texture_groups[f'{sample_texture.entry.chemical_formula}_{sample_texture.preferred_orientation}'] = texture_group_one
                except UnicodeDecodeError:
                    iter_num = iter_num + 1
                    continue
        sample_dup = sample_texture.optimize_by_texture(texture_group_one)    
        first_solution = sample_dup.solution.pop(0)
        sample_dup.solution.append(first_solution)
        sample_texture = Texture(sample_dup,TC_cutoff=1.2)
        iter_num = iter_num + 1
        
    new_samples.append(sample_texture.sample)

In [ ]:
texture_groups = {}
new_samples = []
for i,sample in enumerate(samples):
    iter_num = 1
    print(i)
    sample_texture = Texture(sample,TC_cutoff=1.2)
    while  iter_num <= len(sample.solution):
        if len(sample_texture.preferred_orientation ) != 0:
            print(sample_texture.entry.name)
            print(sample_texture.preferred_orientation)
            if sample_texture.entry.chemical_formula in texture_groups.keys():
                texture_group_one = texture_groups[f'{sample_texture.entry.chemical_formula}_{sample_texture.preferred_orientation}']
            else:
                texture_group_one = sample_texture.get_texture_group()
                texture_groups[f'{sample_texture.entry.chemical_formula}_{sample_texture.preferred_orientation}'] = texture_group_one
                              
        sample_dup = sample_texture.optimize_by_texture(texture_group_one)    
        first_solution = sample_dup.solution.pop(0)
        sample_dup.solution.append(first_solution)
        sample_texture = Texture(sample_dup,TC_cutoff=1.2)
        iter_num = iter_num + 1
        
    new_samples.append(sample_texture.sample)

In [5]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
structure = Structure.from_file('./data/icsd/CollCode22303.cif')
structure
# structure = Structure.from_file('./data/icsd/CollCode22736.cif')

Structure Summary
Lattice
    abc : 9.39 2.93 4.89
 angles : 90.0 90.0 90.0
 volume : 134.537103
      A : 9.39 0.0 5.749716721996824e-16
      B : 4.711799363552546e-16 2.93 1.7941075607508725e-16
      C : 0.0 0.0 4.89
PeriodicSite: V4+ (8.0472, 0.7325, 2.0147) [0.8570, 0.2500, 0.4120]
PeriodicSite: V4+ (3.3522, 0.7325, 0.4303) [0.3570, 0.2500, 0.0880]
PeriodicSite: V4+ (1.3428, 2.1975, 2.8753) [0.1430, 0.7500, 0.5880]
PeriodicSite: V4+ (6.0378, 2.1975, 4.4597) [0.6430, 0.7500, 0.9120]
PeriodicSite: O2- (6.9017, 0.7325, 0.5183) [0.7350, 0.2500, 0.1060]
PeriodicSite: O2- (2.2067, 0.7325, 1.9267) [0.2350, 0.2500, 0.3940]
PeriodicSite: O2- (2.4884, 2.1975, 4.3717) [0.2650, 0.7500, 0.8940]
PeriodicSite: O2- (7.1834, 2.1975, 2.9633) [0.7650, 0.7500, 0.6060]
PeriodicSite: O2- (4.8171, 0.7325, 3.7800) [0.5130, 0.2500, 0.7730]
PeriodicSite: O2- (0.1221, 0.7325, 3.5550) [0.0130, 0.2500, 0.7270]
PeriodicSite: O2- (4.5729, 2.1975, 1.1100) [0.4870, 0.7500, 0.2270]
PeriodicSite: O2- (9.2679, 2.19

In [6]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifParser
structure = CifParser('./data/icsd/CollCode22303.cif').get_structures()[0]
# structure = Structure.from_file('./data/icsd/CollCode22303.cif')
structure
# structure = Structure.from_file('./data/icsd/CollCode22736.cif')

Structure Summary
Lattice
    abc : 2.93 4.89 9.39
 angles : 90.0 90.0 90.0
 volume : 134.537103
      A : 4.711799363552546e-16 2.93 1.7941075607508725e-16
      B : -0.0 -0.0 4.89
      C : 9.39 -0.0 5.749716721996824e-16
PeriodicSite: V4+ (8.0472, 0.7325, 2.0147) [0.2500, 0.4120, 0.8570]
PeriodicSite: V4+ (3.3522, 0.7325, 0.4303) [0.2500, 0.0880, 0.3570]
PeriodicSite: V4+ (1.3428, 2.1975, 2.8753) [0.7500, 0.5880, 0.1430]
PeriodicSite: V4+ (6.0378, 2.1975, 4.4597) [0.7500, 0.9120, 0.6430]
PeriodicSite: O2- (6.9017, 0.7325, 0.5183) [0.2500, 0.1060, 0.7350]
PeriodicSite: O2- (2.2067, 0.7325, 1.9267) [0.2500, 0.3940, 0.2350]
PeriodicSite: O2- (2.4884, 2.1975, 4.3717) [0.7500, 0.8940, 0.2650]
PeriodicSite: O2- (7.1834, 2.1975, 2.9633) [0.7500, 0.6060, 0.7650]
PeriodicSite: O2- (4.8171, 0.7325, 3.7800) [0.2500, 0.7730, 0.5130]
PeriodicSite: O2- (0.1221, 0.7325, 3.5550) [0.2500, 0.7270, 0.0130]
PeriodicSite: O2- (4.5729, 2.1975, 1.1100) [0.7500, 0.2270, 0.4870]
PeriodicSite: O2- (9.2679, 2

In [7]:
for sample in new_samples:    
    solution_file = f'solution/samples{sample.sample_id}.json'
    with open(solution_file, 'w') as f:
        json.dump(sample, f, cls=MontyEncoder)
        sample.plot(perphase=True, saveplot=f'solution_figures/sample_{sample.sample_id}.pdf')